In [2]:
import pandas as pd
import datetime
import PyPDF2
import numpy as np

In [3]:
new_df = pd.read_csv('test.csv',index_col=0)
# add_df = {
#     "Products": ["3_PersonalLoan","2_BusinessLoan","5_AutoLoan"],
#     "Loan Institution":["OHMY Loan Private Limited","Bank123","Bank123"],
#     "date_opened": [datetime.date(2019, 1, 1),datetime.date(2019, 1, 2),datetime.date(2019, 1, 3)],
#     "Sanction/Credit Limit":[10000,12000,15000],
#     "Balance":[100000,200000,300000],
#     "EMI":[1000,2000,3000],
#     "Paid Principle": [21887,21993,22100],
# }
# new_df = new_df.append(add_df, ignore_index=True)
# new_df = pd.concat([new_df,pd.DataFrame(add_df)])
add_df = {
    "Products": ["2_BusinessLoan"],
    "Loan Institution":["OHMY Loan Private Limited"],
    "date_opened": [datetime.date(2019, 1, 1)],
    "Sanction/Credit Limit":[10000],
    "Balance":[100000],
    "EMI":[1000],
    "Paid Principle": [21887],
}
# new_df = new_df.append(add_df, ignore_index=True)
new_df = pd.concat([new_df,pd.DataFrame(add_df)])
new_df =new_df.reset_index()
new_df

,index,Loan Institution,Sanction/Credit Limit,Balance,EMI,Paid Principle,Delinquencies,Products,date_opened
0,3_PersonalLoan,State Bank Of India,961000,962718,19317,0,True,NaN,NaN
1,3_PersonalLoan,State Bank Of India,1,-813858,0,813859,False,NaN,NaN
2,4_Others,Punjab National Bank,800000,805163,0,0,False,NaN,NaN
3,4_Others,Punjab National Bank,80,80,0,0,False,NaN,NaN
4,4_Others,Punjab National Bank,1000000,1039824,9525,0,True,NaN,NaN
5,4_Others,Uttar Bihar Gramin Bank,300000,319508,0,0,False,NaN,NaN
6,4_Others,Central Bank of India,45000,15145,1058,29855,True,NaN,NaN
7,Total,NaN,3106081,2328580,29900,843714,NaN,NaN,NaN
8,0,OHMY Loan Private Limited,10000,100000,1000,21887,NaN,2_BusinessLoan,2019-01-01


In [4]:
# create new_df if lenght of new_df is greater than 3
if len(new_df) > 3:
    new_1 = new_df.iloc[:3]
    new_2 = new_df.iloc[3:]


In [5]:
new_df.sort_values(by=['Paid Principle'],ascending=False,inplace=True)
new_df.shape

(9, 9)

In [6]:
top_up_df = new_df.loc[new_df['Products'].isin(['3_PersonalLoan','5_AutoLoan','6_HouseLoan'])]
# new_df.drop(top_up_df.index)
top_up_df = top_up_df.drop_duplicates(subset=['Loan Institution'],keep='first')
new_df = pd.concat([new_df,top_up_df])
new_df

,index,Loan Institution,Sanction/Credit Limit,Balance,EMI,Paid Principle,Delinquencies,Products,date_opened
7,Total,NaN,3106081,2328580,29900,843714,NaN,NaN,NaN
1,3_PersonalLoan,State Bank Of India,1,-813858,0,813859,False,NaN,NaN
6,4_Others,Central Bank of India,45000,15145,1058,29855,True,NaN,NaN
8,0,OHMY Loan Private Limited,10000,100000,1000,21887,NaN,2_BusinessLoan,2019-01-01
0,3_PersonalLoan,State Bank Of India,961000,962718,19317,0,True,NaN,NaN
2,4_Others,Punjab National Bank,800000,805163,0,0,False,NaN,NaN
3,4_Others,Punjab National Bank,80,80,0,0,False,NaN,NaN
4,4_Others,Punjab National Bank,1000000,1039824,9525,0,True,NaN,NaN
5,4_Others,Uttar Bihar Gramin Bank,300000,319508,0,0,False,NaN,NaN


In [7]:
pivot = pd.pivot_table(top_up_df,values=['Paid Principle',"Balance"],index=['Products'],aggfunc=np.sum,fill_value=0)
pivot

""
Products


In [8]:
def getCases(top_up:int,case_df:dict,pivot:pd.DataFrame):
    global recommendation_string
    balance = pivot['Balance'].to_list()
    if(len(balance) == 0):
        print("No top up")
    else:
        for i in range(len(balance)):
            productBalance = balance[i]
            if(top_up == 0):
                print("No top up")
                break
            elif(productBalance>top_up):
                # REDUCE CONDITION WILL HAVE {REDUCE PRODUCT: OUTSTANDING PRODUCT VALUE}
                print(f"Reduce {pivot.index[i]}")
                tentative_string+=f", Reduce {pivot.index[i]}"
                case_df['Sentence'].append(f"Reduce {pivot.index[i]}")
                outstandingProductBalance = productBalance - top_up
                top_up = 0
                case_df["Value"].append(outstandingProductBalance)
            elif(productBalance<top_up):
                # REMOVE CONDITION WILL HAVE {REMOVE PRODUCT: TOP UP VALUE}
                print(f"Remove {pivot.index[i]}")
                tentative_string+=f", Remove {pivot.index[i]}"
                case_df['Sentence'].append(f"Remove {pivot.index[i]}")
                top_up = top_up - productBalance
                productBalance = 0
                case_df['Value'].append(top_up)
            else:
                print("No recommendation")
        if(len(tentative_string)>0):
            recommendation_string+=" We recommend you use this to "+tentative_string
        return case_df

In [9]:
# pivot = pd.pivot_table(new_df,values=['Paid Principle',"Balance"],index=['Products'],aggfunc=np.sum,fill_value=0)
# # test = pivot[pivot.index == '3_PersonalLoan']
# # pivot = pivot.drop(test.index)
# top_up_list = []
# try:
#     top_up_list.append(np.where(pivot.index == '3_PersonalLoan')[0][0])
# except IndexError:
#     pass
# try:
#     top_up_list.append(np.where(pivot.index == '5_AutoLoan')[0][0])
# except IndexError:
#     pass
# try:
#     top_up_list.append(np.where(pivot.index == '6_HouseLoan')[0][0])
# except IndexError:
#     pass
# pivot.loc["1_CreditCard","Balance"] = 10
# pivot = pivot.reset_index()
# pivot['Products'] = pivot['Products'].map({
#     '1_CreditCard':'Credit Card',
#     '2_BusinessLoan':'Business Loan',
#     '3_PersonalLoan':'Personal Loan',
#     '4_Others':'Others',
#     '5_AutoLoan':'Auto Loan',
#     '6_HouseLoan':'House Loan',
# })
# pivot = pivot.dropna(subset=['Products'],axis=0)
# pivot = pivot.set_index('Products')
# top_up_list.insert(0,0)
# case_df = {"Sentence":[],"Value":[]}
# products = [pivot.index[x] for x in top_up_list]
recommendation_string = ""
# if(len(top_up_list) > 0):
#     recommendation+="You can get a top up on "+', '.join(products)+" based on the amount you have already paid for. We recommend you use this to "
#     for i in range(1,len(top_up_list)):
#         top_up_index = top_up_list[i]
#         top_up = pivot.loc[pivot.index[top_up_index]]['Paid Principle']
#         if(i==1):
#             local_pivot = pivot.loc[pivot.index[top_up_list[i-1]:top_up_list[i]]]
#         else:
#             local_pivot = pivot.loc[pivot.index[top_up_list[i-1]+1:top_up_list[i]]]
#         case_df = getCases(top_up=top_up,case_df =case_df,pivot =local_pivot)
# else:
#     print("Not eligible for top up")
# case_df = pd.DataFrame(case_df)
# recommendation

In [10]:
# emi must be greater than 12 months
# new_df = new_df.drop(new_df[(new_df['date_diff'] < 12) & (new_df['Products'].isin(['3_PersonalLoan','5_AutoLoan','6_HouseLoan']))].index)
new_df.sort_values(by=['Paid Principle'],ascending=False,inplace = True)
top_up_df = new_df.loc[new_df['Products'].isin(['3_PersonalLoan','5_AutoLoan','6_HouseLoan'])]
new_df.drop(top_up_df.index,inplace=True)
top_up_df = top_up_df.drop_duplicates(subset=['Loan Institution'],keep='first')
new_df = pd.concat([new_df,top_up_df])
pivot = pd.pivot_table(top_up_df,values=['Paid Principle',"Balance"],index=['Products'],aggfunc=np.sum,fill_value=0)
top_up_list = []
try:
    top_up_list.append(np.where(pivot.index == '3_PersonalLoan')[0][0])
except IndexError:
    pass
try:
    top_up_list.append(np.where(pivot.index == '5_AutoLoan')[0][0])
except IndexError:
    pass
try:
    top_up_list.append(np.where(pivot.index == '6_HouseLoan')[0][0])
except IndexError:
    pass
pivot = pivot.reset_index()
pivot['Products'] = pivot['Products'].map({
    '1_CreditCard':'Credit Card',
    '2_BusinessLoan':'Business Loan',
    '3_PersonalLoan':'Personal Loan',
    '4_Others':'Others',
    '5_AutoLoan':'Auto Loan',
    '6_HouseLoan':'House Loan',
})
# pivot = pivot.dropna(subset=['Products'],axis=0)
pivot = pivot.set_index('Products')
top_up_list.insert(0,0)
case_df = {"Sentence":[],"Value":[]}
try:
    products = set([pivot.index[x] for x in top_up_list])
    if(len(top_up_list) > 0):
        for i in range(1,len(top_up_list)):
            top_up_index = top_up_list[i]
            top_up = pivot.loc[pivot.index[top_up_index]]['Paid Principle']
            if(i==1):
                local_pivot = pivot.loc[pivot.index[top_up_list[i-1]:top_up_list[i]]]
            else:
                local_pivot = pivot.loc[pivot.index[top_up_list[i-1]+1:top_up_list[i]]]
            case_df = getCases(top_up=top_up,case_df =case_df,pivot =local_pivot)
    else:
        recommendation_string+=""
    print(recommendation_string)
    case_df = pd.DataFrame(case_df)
    case_df
except IndexError:
    pass
recommendation_string+="You can get a top up on "+', '.join(products)+" based on the amount you have already paid for. "

NameError: name 'products' is not defined

In [ ]:
# Recommendation above 650